# Extract- Connecting to database servers

In [1]:
import numpy as np
import pandas as pd
import pymysql
import psycopg2
import warnings
warnings.filterwarnings('ignore')

## Data from postgreSQL

In [2]:
postgres_connection = psycopg2.connect(
    user='*****',
    password='*****',
    host='localhost',
    database='TruckDelay',
    port='5432')

In [3]:
routes_df = pd.read_sql_query('select * from routes_details', postgres_connection)

In [4]:
routes_df.head(2)

,route_id,origin_id,destination_id,distance,average_hours
0,R-ada2a391,C-927ceb5e,C-56e39a5e,1735.06,34.70
1,R-ae0ef31f,C-927ceb5e,C-73ae5412,1498.24,29.96


In [5]:
routes_weather = pd.read_sql_query('select * from routes_weather', postgres_connection)

In [6]:
routes_weather.head(2)

,route_id,Date,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,R-ada2a391,2019-01-01 00:00:00,30.0,11.0,Heavy snow,0.0,90,1.0,1010,0,0,0,0
1,R-ada2a391,2019-01-01 06:00:00,30.0,11.0,Heavy snow,0.0,91,3.0,1012,0,0,0,0


In [7]:
postgres_connection.close()

## Data from MySQL

In [8]:
mysql_connection = pymysql.connect(
    user='*****',
    passwd='*****',
    db='truck_delay',
    host='127.0.0.1',
    port=3306)

In [9]:
pd.read_sql_query('show tables',mysql_connection)

,Tables_in_truck_delay
0,city_weather
1,drivers_details
2,traffic_details
3,truck_details
4,truck_schedule_data


In [10]:
truck_df = pd.read_sql_query('select * from truck_details',mysql_connection)
traffic_df = pd.read_sql_query('select * from traffic_details',mysql_connection)
driver_df = pd.read_sql_query('select * from drivers_details',mysql_connection)
weather_df = pd.read_sql_query('select * from city_weather',mysql_connection)
schedule_df = pd.read_sql_query('select * from truck_schedule_data',mysql_connection)

In [11]:
truck_df.head(2)

,truck_id,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,42302347,10,3000.0,17,gas
1,27867488,14,10000.0,22,diesel


In [12]:
traffic_df.head(2)

,route_id,date,hour,no_of_vehicles,accident
0,R-ada2a391,2019-01-01,0,669.0,0
1,R-ada2a391,2019-01-01,100,628.0,0


In [13]:
driver_df.head(2)

,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph
0,d9f30553-6,Daniel Marks,male,47,5,proactive,7,42302347,62.22
1,82de7bb8-2,Clifford Carr,male,47,14,proactive,4,27867488,60.89


In [14]:
weather_df.head(2)

,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,C-927ceb5e,2019-01-01,0,30.0,11.0,Light snow,0.0,86,6.0,1019.0,0.0,0.0,0.0,0.0
1,C-927ceb5e,2019-01-01,100,28.0,12.0,Light snow,0.0,86,5.0,1021.0,0.0,0.0,0.0,0.0


In [15]:
schedule_df.head(2)

,truck_id,route_id,departure_date,estimated_arrival,delay
0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12.,0
1,59856374,R-29ea762e,2019-01-01 07:00:00,2019-01-02 04:01:12.,0


# Transform : Data Cleaning, Feature Engineering and Merging

## Data Cleaning

To avoid data leakage missing values will be treated after train test split

In [16]:
# Lets take the wether data and drop uncecessary columns
weather_df.head(2)

,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,C-927ceb5e,2019-01-01,0,30.0,11.0,Light snow,0.0,86,6.0,1019.0,0.0,0.0,0.0,0.0
1,C-927ceb5e,2019-01-01,100,28.0,12.0,Light snow,0.0,86,5.0,1021.0,0.0,0.0,0.0,0.0


In [17]:
weather_df.drop(columns=['chanceofrain','chanceoffog','chanceofsnow','chanceofthunder'],inplace=True)

In [18]:
weather_df.head(2)

,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure
0,C-927ceb5e,2019-01-01,0,30.0,11.0,Light snow,0.0,86,6.0,1019.0
1,C-927ceb5e,2019-01-01,100,28.0,12.0,Light snow,0.0,86,5.0,1021.0


In [19]:
# convert hour to 4 digit string format
weather_df['hour'] = weather_df['hour'].apply(lambda x: f'{x:04d}')

# convert hour to time format
weather_df['hour'] = pd.to_datetime(weather_df['hour'],format='%H%M').dt.time

In [20]:
weather_df.head(2)

,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure
0,C-927ceb5e,2019-01-01,00:00:00,30.0,11.0,Light snow,0.0,86,6.0,1019.0
1,C-927ceb5e,2019-01-01,01:00:00,28.0,12.0,Light snow,0.0,86,5.0,1021.0


In [21]:
# Lets combine hour and date and create a new column custom date

custom_date = pd.to_datetime(weather_df['date'].astype('str')+' '+weather_df['hour'].astype('str'))

In [22]:
weather_df.insert(loc=1,column='custom_date',value=custom_date)

In [23]:
weather_df.head(2)

,city_id,custom_date,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure
0,C-927ceb5e,2019-01-01 00:00:00,2019-01-01,00:00:00,30.0,11.0,Light snow,0.0,86,6.0,1019.0
1,C-927ceb5e,2019-01-01 01:00:00,2019-01-01,01:00:00,28.0,12.0,Light snow,0.0,86,5.0,1021.0


In [24]:
# Now lets remove unnecesary columns from route wether
routes_weather.head(2)

,route_id,Date,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,R-ada2a391,2019-01-01 00:00:00,30.0,11.0,Heavy snow,0.0,90,1.0,1010,0,0,0,0
1,R-ada2a391,2019-01-01 06:00:00,30.0,11.0,Heavy snow,0.0,91,3.0,1012,0,0,0,0


In [25]:
routes_weather.drop(columns=['chanceofrain','chanceoffog','chanceofsnow','chanceofthunder'],inplace=True)

In [26]:
routes_weather.head(2)

,route_id,Date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-ada2a391,2019-01-01 00:00:00,30.0,11.0,Heavy snow,0.0,90,1.0,1010
1,R-ada2a391,2019-01-01 06:00:00,30.0,11.0,Heavy snow,0.0,91,3.0,1012


In [27]:
# Now let us look at traffic data

In [28]:
traffic_df.head(2)

,route_id,date,hour,no_of_vehicles,accident
0,R-ada2a391,2019-01-01,0,669.0,0
1,R-ada2a391,2019-01-01,100,628.0,0


In [29]:
# create custom date column
traffic_df['hour'] = traffic_df['hour'].apply(lambda x: f'{x:04d}')

traffic_df['hour'] = pd.to_datetime(traffic_df['hour'],format='%H%M').dt.time

In [30]:
custom_date = pd.to_datetime(traffic_df['date'].astype('str')+' '+traffic_df['hour'].astype('str'))

In [31]:
traffic_df.insert(1,'custom_date',custom_date)

In [32]:
traffic_df.head(2)

,route_id,custom_date,date,hour,no_of_vehicles,accident
0,R-ada2a391,2019-01-01 00:00:00,2019-01-01,00:00:00,669.0,0
1,R-ada2a391,2019-01-01 01:00:00,2019-01-01,01:00:00,628.0,0


## Feature Engineering

In [33]:
# Merge route wether with schedule data

In [34]:
routes_weather.head(3)

,route_id,Date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-ada2a391,2019-01-01 00:00:00,30.0,11.0,Heavy snow,0.0,90,1.0,1010
1,R-ada2a391,2019-01-01 06:00:00,30.0,11.0,Heavy snow,0.0,91,3.0,1012
2,R-ada2a391,2019-01-02 00:00:00,28.0,11.0,Cloudy,0.0,91,4.0,1013


In [35]:
schedule_df.head(3)

,truck_id,route_id,departure_date,estimated_arrival,delay
0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12.,0
1,59856374,R-29ea762e,2019-01-01 07:00:00,2019-01-02 04:01:12.,0
2,12602955,R-a3d67783,2019-01-01 07:00:00,2019-01-01 07:45:36.,0


In [36]:
# Add unique value to rows in schedule df
schedule_df.insert(0,'unique_id',np.arange(len(schedule_df)))

In [37]:
schedule_df['estimated_arrival']=pd.to_datetime(schedule_df['estimated_arrival'])

In [38]:
schedule_df_nearest_6h = schedule_df.copy()

In [39]:
schedule_df_nearest_6h['departure_date']=schedule_df_nearest_6h['departure_date'].dt.floor('6H')
schedule_df_nearest_6h['estimated_arrival']=schedule_df_nearest_6h['estimated_arrival'].dt.ceil('6H')

In [40]:
schedule_df_nearest_6h

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay
0,0,30312694,R-b236e347,2019-01-01 06:00:00,2019-01-01 18:00:00,0
1,1,59856374,R-29ea762e,2019-01-01 06:00:00,2019-01-02 06:00:00,0
2,2,12602955,R-a3d67783,2019-01-01 06:00:00,2019-01-01 12:00:00,0
3,3,46619422,R-31ec9310,2019-01-01 06:00:00,2019-01-02 00:00:00,0
4,4,10140178,R-a07c5dbd,2019-01-01 06:00:00,2019-01-02 00:00:00,0
...,...,...,...,...,...,...
12303,12303,31047945,R-1484a7ea,2019-02-06 06:00:00,2019-02-14 00:00:00,0
12304,12304,14758432,R-927cf900,2019-02-06 06:00:00,2019-02-14 00:00:00,1
12305,12305,31370619,R-5a83ad98,2019-02-06 06:00:00,2019-02-14 12:00:00,1
12306,12306,67332883,R-991530bc,2019-02-06 06:00:00,2019-02-13 12:00:00,1


In [41]:
date_6h_range = [pd.date_range(start,end,freq='6H') for start,end in 
 zip(schedule_df_nearest_6h['departure_date'], schedule_df_nearest_6h['estimated_arrival'])]

In [42]:
schedule_df_nearest_6h['Date']= date_6h_range

In [43]:
schedule_df_exploded_6h = schedule_df_nearest_6h.explode('Date',ignore_index=False)

In [44]:
schedule_df_exploded_6h.head(3)

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,Date
0,0,30312694,R-b236e347,2019-01-01 06:00:00,2019-01-01 18:00:00,0,2019-01-01 06:00:00
0,0,30312694,R-b236e347,2019-01-01 06:00:00,2019-01-01 18:00:00,0,2019-01-01 12:00:00
0,0,30312694,R-b236e347,2019-01-01 06:00:00,2019-01-01 18:00:00,0,2019-01-01 18:00:00


In [45]:
schedule_df_exploded_6h.shape

(71376, 7)

In [46]:
schedule_weather = schedule_df_exploded_6h.merge(routes_weather,on=['Date','route_id'],how='left')

In [47]:
schedule_weather.head(3)

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,Date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,0,30312694,R-b236e347,2019-01-01 06:00:00,2019-01-01 18:00:00,0,2019-01-01 06:00:00,39.0,5.0,Light rain,0.0,98,6.0,1008
1,0,30312694,R-b236e347,2019-01-01 06:00:00,2019-01-01 18:00:00,0,2019-01-01 12:00:00,23.0,4.0,Clear,0.0,76,6.0,1011
2,0,30312694,R-b236e347,2019-01-01 06:00:00,2019-01-01 18:00:00,0,2019-01-01 18:00:00,23.0,4.0,Clear,0.0,73,6.0,1011


In [48]:
schedule_weather.isnull().sum()

unique_id            0
truck_id             0
route_id             0
departure_date       0
estimated_arrival    0
delay                0
Date                 0
temp                 0
wind_speed           0
description          0
precip               0
humidity             0
visibility           0
pressure             0
dtype: int64

In [49]:
schedule_weather_grp = schedule_weather.groupby(by=['unique_id','truck_id','route_id']).agg(
    route_avg_temp = ('temp','mean'),
    route_avg_wind = ('wind_speed','mean'),
    route_description = ('description',(lambda x: x.mode()[0])),
    route_avg_precip = ('precip','mean'),
    route_avg_humidity = ('humidity','mean'),
    route_avg_visibility = ('visibility','mean'),
    route_avg_pressure = ('pressure','mean'))

                         

In [50]:
schedule_weather_merge = schedule_df.merge(schedule_weather_grp,on=['unique_id','truck_id','route_id'],how='left')

In [51]:
schedule_weather_merge.head(3)

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind,route_description,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure
0,0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12,0,28.333333,4.333333,Clear,0.0,82.333333,6.0,1010.0
1,1,59856374,R-29ea762e,2019-01-01 07:00:00,2019-01-02 04:01:12,0,40.000000,10.600000,Overcast,0.0,78.800000,6.0,1022.2
2,2,12602955,R-a3d67783,2019-01-01 07:00:00,2019-01-01 07:45:36,0,24.000000,7.500000,Clear,0.0,63.500000,6.0,1020.0


In [52]:
schedule_weather_merge.isnull().sum()

unique_id               0
truck_id                0
route_id                0
departure_date          0
estimated_arrival       0
delay                   0
route_avg_temp          0
route_avg_wind          0
route_description       0
route_avg_precip        0
route_avg_humidity      0
route_avg_visibility    0
route_avg_pressure      0
dtype: int64

Creating destination and source city weather

In [53]:
weather_df.head(3)

,city_id,custom_date,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure
0,C-927ceb5e,2019-01-01 00:00:00,2019-01-01,00:00:00,30.0,11.0,Light snow,0.0,86,6.0,1019.0
1,C-927ceb5e,2019-01-01 01:00:00,2019-01-01,01:00:00,28.0,12.0,Light snow,0.0,86,5.0,1021.0
2,C-927ceb5e,2019-01-01 02:00:00,2019-01-01,02:00:00,28.0,13.0,Moderate snow,0.0,85,4.0,1022.0


In [54]:
routes_df.head(3)

,route_id,origin_id,destination_id,distance,average_hours
0,R-ada2a391,C-927ceb5e,C-56e39a5e,1735.06,34.70
1,R-ae0ef31f,C-927ceb5e,C-73ae5412,1498.24,29.96
2,R-4beec5fd,C-927ceb5e,C-4fe0fa24,6078.46,121.57


In [55]:
schedule_weather_merge.merge(routes_df,on=['route_id'],how='left')

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind,route_description,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,origin_id,destination_id,distance,average_hours
0,0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12,0,28.333333,4.333333,Clear,0.000000,82.333333,6.000000,1010.000000,C-927ceb5e,C-d80a1e7d,310.75,6.22
1,1,59856374,R-29ea762e,2019-01-01 07:00:00,2019-01-02 04:01:12,0,40.000000,10.600000,Overcast,0.000000,78.800000,6.000000,1022.200000,C-927ceb5e,C-639c5e36,1050.84,21.02
2,2,12602955,R-a3d67783,2019-01-01 07:00:00,2019-01-01 07:45:36,0,24.000000,7.500000,Clear,0.000000,63.500000,6.000000,1020.000000,C-927ceb5e,C-a9f2c329,38.10,0.76
3,3,46619422,R-31ec9310,2019-01-01 07:00:00,2019-01-01 20:46:48,0,40.250000,5.250000,Clear,0.000000,84.500000,6.000000,1011.250000,C-927ceb5e,C-c7cacd1d,688.91,13.78
4,4,10140178,R-a07c5dbd,2019-01-01 07:00:00,2019-01-01 21:34:11,0,19.500000,4.500000,Fog,0.000000,99.500000,2.500000,1015.000000,C-927ceb5e,C-ef47bdcd,728.40,14.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,12303,31047945,R-1484a7ea,2019-02-06 07:00:00,2019-02-13 22:40:12,0,60.500000,7.625000,Light rain shower,0.037500,93.312500,4.312500,1007.187500,C-34ae77b0,C-731988ba,9183.35,183.67
12304,12304,14758432,R-927cf900,2019-02-06 07:00:00,2019-02-13 18:52:48,1,53.500000,4.218750,Sunny,0.012500,57.625000,5.687500,1013.093750,C-34ae77b0,C-a9f2c329,8994.17,179.88
12305,12305,31370619,R-5a83ad98,2019-02-06 07:00:00,2019-02-14 08:01:48,1,61.029412,4.382353,Sunny,0.000000,80.058824,5.235294,1012.558824,C-34ae77b0,C-c7cacd1d,9651.62,193.03
12306,12306,67332883,R-991530bc,2019-02-06 07:00:00,2019-02-13 11:14:24,1,45.866667,7.533333,Patchy light snow,0.000000,72.233333,4.500000,1023.233333,C-34ae77b0,C-84f378bb,8611.88,172.24


In [56]:
weather_df

,city_id,custom_date,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure
0,C-927ceb5e,2019-01-01 00:00:00,2019-01-01,00:00:00,30.0,11.0,Light snow,0.0,86,6.0,1019.0
1,C-927ceb5e,2019-01-01 01:00:00,2019-01-01,01:00:00,28.0,12.0,Light snow,0.0,86,5.0,1021.0
2,C-927ceb5e,2019-01-01 02:00:00,2019-01-01,02:00:00,28.0,13.0,Moderate snow,0.0,85,4.0,1022.0
3,C-927ceb5e,2019-01-01 03:00:00,2019-01-01,03:00:00,28.0,14.0,Moderate snow,0.0,84,3.0,1024.0
4,C-927ceb5e,2019-01-01 04:00:00,2019-01-01,04:00:00,28.0,13.0,Moderate snow,0.0,84,3.0,1025.0
...,...,...,...,...,...,...,...,...,...,...,...
55171,C-594514f8,2019-02-15 19:00:00,2019-02-15,19:00:00,23.0,3.0,Cloudy,0.0,74,6.0,1018.0
55172,C-594514f8,2019-02-15 20:00:00,2019-02-15,20:00:00,23.0,3.0,Light snow,0.0,74,6.0,1018.0
55173,C-594514f8,2019-02-15 21:00:00,2019-02-15,21:00:00,23.0,3.0,Light snow,0.0,75,6.0,1019.0
55174,C-594514f8,2019-02-15 22:00:00,2019-02-15,22:00:00,23.0,2.0,Light snow,0.0,75,6.0,1019.0


In [57]:
schedule_wether_copy = schedule_weather_merge.copy()

In [58]:
schedule_wether_copy = schedule_wether_copy.merge(routes_df,on='route_id',how='left')

In [59]:
schedule_wether_copy['departure_date'] = schedule_wether_copy['departure_date'].dt.round('1h')
schedule_wether_copy['estimated_arrival']=schedule_wether_copy['estimated_arrival'].dt.round('1h')

In [60]:
date_range = [pd.date_range(start,stop,freq='1h') for start,stop in
             zip(schedule_wether_copy['departure_date'],schedule_wether_copy['estimated_arrival'])]

In [61]:
schedule_wether_copy['date']=date_range

In [62]:
schedule_wether_copy = schedule_wether_copy.explode('date')

In [63]:
schedule_wether_city_copy = schedule_wether_copy.merge(weather_df,
                           left_on=['origin_id','date'],right_on=['city_id','custom_date'],
                           how='left')

In [64]:
schedule_wether_city_copy.columns

Index(['unique_id', 'truck_id', 'route_id', 'departure_date',
       'estimated_arrival', 'delay', 'route_avg_temp', 'route_avg_wind',
       'route_description', 'route_avg_precip', 'route_avg_humidity',
       'route_avg_visibility', 'route_avg_pressure', 'origin_id',
       'destination_id', 'distance', 'average_hours', 'date_x', 'city_id',
       'custom_date', 'date_y', 'hour', 'temp', 'wind_speed', 'description',
       'precip', 'humidity', 'visibility', 'pressure'],
      dtype='object')

In [65]:
origin_wether_df = schedule_wether_city_copy.groupby(by=['route_id','origin_id'],as_index=False).agg(
    origin_avg_temp = ('temp','mean'),
    origin_avg_wind = ('wind_speed','mean'),
    origin_description = ('description',(lambda x: x.mode()[0])),
    origin_avg_precip = ('precip','mean'),
    origin_avg_humidity = ('humidity','mean'),
    origin_avg_visibility = ('visibility','mean'),
    origin_avg_pressure = ('pressure','mean'))

In [66]:
origin_wether_df.isnull().sum()

route_id                 0
origin_id                0
origin_avg_temp          0
origin_avg_wind          0
origin_description       0
origin_avg_precip        0
origin_avg_humidity      0
origin_avg_visibility    0
origin_avg_pressure      0
dtype: int64

In [67]:
schedule_wether_city_copy = schedule_wether_copy.merge(weather_df,
                           left_on=['destination_id','date'],right_on=['city_id','custom_date'],
                           how='left')

In [68]:
destination_wether_df = schedule_wether_city_copy.groupby(by=['route_id','destination_id'],as_index=False).agg(
    dest_avg_temp = ('temp','mean'),
    dest_avg_wind = ('wind_speed','mean'),
    dest_description = ('description',(lambda x: x.mode()[0])),
    dest_avg_precip = ('precip','mean'),
    dest_avg_humidity = ('humidity','mean'),
    dest_avg_visibility = ('visibility','mean'),
    dest_avg_pressure = ('pressure','mean'))

In [69]:
destination_wether_df.isnull().sum()

route_id               0
destination_id         0
dest_avg_temp          0
dest_avg_wind          0
dest_description       0
dest_avg_precip        0
dest_avg_humidity      0
dest_avg_visibility    0
dest_avg_pressure      0
dtype: int64

In [70]:
# merge schedule_wether, origin_wether and destination_wether

In [71]:
schedule_weather_origin_merge = schedule_weather_merge.merge(origin_wether_df,on='route_id',how='left')

In [72]:
schedule_weather_origin_dest_merge = schedule_weather_origin_merge.merge(destination_wether_df,
                                                                         on='route_id',how='left')

In [73]:
schedule_weather_origin_dest_merge.columns

Index(['unique_id', 'truck_id', 'route_id', 'departure_date',
       'estimated_arrival', 'delay', 'route_avg_temp', 'route_avg_wind',
       'route_description', 'route_avg_precip', 'route_avg_humidity',
       'route_avg_visibility', 'route_avg_pressure', 'origin_id',
       'origin_avg_temp', 'origin_avg_wind', 'origin_description',
       'origin_avg_precip', 'origin_avg_humidity', 'origin_avg_visibility',
       'origin_avg_pressure', 'destination_id', 'dest_avg_temp',
       'dest_avg_wind', 'dest_description', 'dest_avg_precip',
       'dest_avg_humidity', 'dest_avg_visibility', 'dest_avg_pressure'],
      dtype='object')

In [74]:
# Next we need to merge traffic_df

In [75]:
swod_merge_copy = schedule_weather_origin_dest_merge.copy()

In [76]:
swod_merge_copy['departure_date'] = swod_merge_copy['departure_date'].dt.round('1h')
swod_merge_copy['estimated_arrival'] = swod_merge_copy['estimated_arrival'].dt.round('1h')

In [77]:
date_range = [pd.date_range(start,stop,freq='1h') for start,stop in
              zip(swod_merge_copy['departure_date'],swod_merge_copy['estimated_arrival'])]

In [78]:
swod_merge_copy['custom_date'] = date_range

In [79]:
swod_merge_copy = swod_merge_copy.explode('custom_date')

In [80]:
swod_merge_copy

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind,route_description,route_avg_precip,...,origin_avg_pressure,destination_id,dest_avg_temp,dest_avg_wind,dest_description,dest_avg_precip,dest_avg_humidity,dest_avg_visibility,dest_avg_pressure,custom_date
0,0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:00:00,0,28.333333,4.333333,Clear,0.000000,...,1020.892857,C-d80a1e7d,31.214286,9.392857,Light drizzle,0.007143,78.000000,4.375000,1018.785714,2019-01-01 07:00:00
0,0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:00:00,0,28.333333,4.333333,Clear,0.000000,...,1020.892857,C-d80a1e7d,31.214286,9.392857,Light drizzle,0.007143,78.000000,4.375000,1018.785714,2019-01-01 08:00:00
0,0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:00:00,0,28.333333,4.333333,Clear,0.000000,...,1020.892857,C-d80a1e7d,31.214286,9.392857,Light drizzle,0.007143,78.000000,4.375000,1018.785714,2019-01-01 09:00:00
0,0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:00:00,0,28.333333,4.333333,Clear,0.000000,...,1020.892857,C-d80a1e7d,31.214286,9.392857,Light drizzle,0.007143,78.000000,4.375000,1018.785714,2019-01-01 10:00:00
0,0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:00:00,0,28.333333,4.333333,Clear,0.000000,...,1020.892857,C-d80a1e7d,31.214286,9.392857,Light drizzle,0.007143,78.000000,4.375000,1018.785714,2019-01-01 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12307,12307,26630473,R-3f49bd43,2019-02-06 07:00:00,2019-02-14 03:00:00,1,57.424242,10.757576,Sunny,0.009091,...,1009.783069,C-c4565ee8,36.507937,8.671958,Sunny,0.010582,74.161376,4.891534,1019.277778,2019-02-13 23:00:00
12307,12307,26630473,R-3f49bd43,2019-02-06 07:00:00,2019-02-14 03:00:00,1,57.424242,10.757576,Sunny,0.009091,...,1009.783069,C-c4565ee8,36.507937,8.671958,Sunny,0.010582,74.161376,4.891534,1019.277778,2019-02-14 00:00:00
12307,12307,26630473,R-3f49bd43,2019-02-06 07:00:00,2019-02-14 03:00:00,1,57.424242,10.757576,Sunny,0.009091,...,1009.783069,C-c4565ee8,36.507937,8.671958,Sunny,0.010582,74.161376,4.891534,1019.277778,2019-02-14 01:00:00
12307,12307,26630473,R-3f49bd43,2019-02-06 07:00:00,2019-02-14 03:00:00,1,57.424242,10.757576,Sunny,0.009091,...,1009.783069,C-c4565ee8,36.507937,8.671958,Sunny,0.010582,74.161376,4.891534,1019.277778,2019-02-14 02:00:00


In [81]:
swodt_merge_copy = swod_merge_copy.merge(traffic_df,on=['route_id','custom_date'],how='left')

In [82]:
traffic_data = swodt_merge_copy.groupby(by='route_id',as_index=False).agg(
    avg_nov = ('no_of_vehicles','mean'),
    accident = ('accident',(lambda x: 1 if any(x==1) else 0)))

In [83]:
swodt_merge = schedule_weather_origin_dest_merge.merge(traffic_data,on='route_id',
                                                      how='left')

In [84]:
swodt_merge

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind,route_description,route_avg_precip,...,destination_id,dest_avg_temp,dest_avg_wind,dest_description,dest_avg_precip,dest_avg_humidity,dest_avg_visibility,dest_avg_pressure,avg_nov,accident
0,0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12,0,28.333333,4.333333,Clear,0.000000,...,C-d80a1e7d,31.214286,9.392857,Light drizzle,0.007143,78.000000,4.375000,1018.785714,1932.714286,1
1,1,59856374,R-29ea762e,2019-01-01 07:00:00,2019-01-02 04:01:12,0,40.000000,10.600000,Overcast,0.000000,...,C-639c5e36,17.306818,6.085227,Partly cloudy,0.000000,83.329545,5.306818,1027.062500,1432.937500,1
2,2,12602955,R-a3d67783,2019-01-01 07:00:00,2019-01-01 07:45:36,0,24.000000,7.500000,Clear,0.000000,...,C-a9f2c329,19.937500,8.437500,Blizzard,0.006250,88.000000,4.000000,1020.375000,1763.250000,1
3,3,46619422,R-31ec9310,2019-01-01 07:00:00,2019-01-01 20:46:48,0,40.250000,5.250000,Clear,0.000000,...,C-c7cacd1d,21.600000,10.383333,Heavy snow,0.015000,79.708333,3.875000,1020.258333,1707.316667,1
4,4,10140178,R-a07c5dbd,2019-01-01 07:00:00,2019-01-01 21:34:11,0,19.500000,4.500000,Fog,0.000000,...,C-ef47bdcd,28.421875,8.351562,Sunny,0.000000,54.289062,5.851562,1020.062500,1688.554688,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,12303,31047945,R-1484a7ea,2019-02-06 07:00:00,2019-02-13 22:40:12,0,60.500000,7.625000,Light rain shower,0.037500,...,C-731988ba,66.997297,3.743243,Clear,0.000000,58.554054,6.000000,1012.751351,1844.054054,1
12304,12304,14758432,R-927cf900,2019-02-06 07:00:00,2019-02-13 18:52:48,1,53.500000,4.218750,Sunny,0.012500,...,C-a9f2c329,28.994475,9.436464,Sunny,0.004420,78.743094,4.994475,1019.604972,1839.093923,1
12305,12305,31370619,R-5a83ad98,2019-02-06 07:00:00,2019-02-14 08:01:48,1,61.029412,4.382353,Sunny,0.000000,...,C-c7cacd1d,30.280928,10.427835,Partly cloudy,0.006186,74.994845,4.337629,1018.030928,1802.474227,1
12306,12306,67332883,R-991530bc,2019-02-06 07:00:00,2019-02-13 11:14:24,1,45.866667,7.533333,Patchy light snow,0.000000,...,C-84f378bb,35.716763,8.234104,Sunny,0.002023,62.476879,5.705202,1021.274566,1813.329480,1


In [85]:
# Lets merge truck_df and driver_df
swodt_truck_merge = swodt_merge.merge(truck_df,on='truck_id',how='left')

In [86]:
final_data = swodt_truck_merge.merge(driver_df,left_on=['truck_id'],right_on=['vehicle_no'],
                        how='left').drop(columns=['vehicle_no'])

In [87]:
final_data

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind,route_description,route_avg_precip,...,mileage_mpg,fuel_type,driver_id,name,gender,age,experience,driving_style,ratings,average_speed_mph
0,0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12,0,28.333333,4.333333,Clear,0.000000,...,23,diesel,94ed3e6d-f,Manuel Wise,male,48,9,proactive,7,57.36
1,1,59856374,R-29ea762e,2019-01-01 07:00:00,2019-01-02 04:01:12,0,40.000000,10.600000,Overcast,0.000000,...,21,diesel,0efc20ea-5,Steven Bishop,male,47,12,proactive,8,57.04
2,2,12602955,R-a3d67783,2019-01-01 07:00:00,2019-01-01 07:45:36,0,24.000000,7.500000,Clear,0.000000,...,26,diesel,8b5ca0fd-7,Nathan Baker,male,43,6,proactive,8,64.11
3,3,46619422,R-31ec9310,2019-01-01 07:00:00,2019-01-01 20:46:48,0,40.250000,5.250000,Clear,0.000000,...,27,diesel,4d8521f5-6,Andrew Thompson,male,49,16,conservative,4,49.47
4,4,10140178,R-a07c5dbd,2019-01-01 07:00:00,2019-01-01 21:34:11,0,19.500000,4.500000,Fog,0.000000,...,23,diesel,d809c9ce-3,Joel Phillips,male,56,22,conservative,9,43.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,12303,31047945,R-1484a7ea,2019-02-06 07:00:00,2019-02-13 22:40:12,0,60.500000,7.625000,Light rain shower,0.037500,...,28,diesel,2a819968-b,Brian Vaughn,male,58,11,conservative,4,39.89
12304,12304,14758432,R-927cf900,2019-02-06 07:00:00,2019-02-13 18:52:48,1,53.500000,4.218750,Sunny,0.012500,...,21,diesel,cc4226d8-b,John Zhang,male,47,16,proactive,7,63.88
12305,12305,31370619,R-5a83ad98,2019-02-06 07:00:00,2019-02-14 08:01:48,1,61.029412,4.382353,Sunny,0.000000,...,18,gas,117112bb-8,Victoria Smith,female,56,21,proactive,8,60.14
12306,12306,67332883,R-991530bc,2019-02-06 07:00:00,2019-02-13 11:14:24,1,45.866667,7.533333,Patchy light snow,0.000000,...,22,diesel,f48cbc8a-c,James Nichols,male,37,7,conservative,3,36.44


In [88]:
final_data.columns

Index(['unique_id', 'truck_id', 'route_id', 'departure_date',
       'estimated_arrival', 'delay', 'route_avg_temp', 'route_avg_wind',
       'route_description', 'route_avg_precip', 'route_avg_humidity',
       'route_avg_visibility', 'route_avg_pressure', 'origin_id',
       'origin_avg_temp', 'origin_avg_wind', 'origin_description',
       'origin_avg_precip', 'origin_avg_humidity', 'origin_avg_visibility',
       'origin_avg_pressure', 'destination_id', 'dest_avg_temp',
       'dest_avg_wind', 'dest_description', 'dest_avg_precip',
       'dest_avg_humidity', 'dest_avg_visibility', 'dest_avg_pressure',
       'avg_nov', 'accident', 'truck_age', 'load_capacity_pounds',
       'mileage_mpg', 'fuel_type', 'driver_id', 'name', 'gender', 'age',
       'experience', 'driving_style', 'ratings', 'average_speed_mph'],
      dtype='object')

In [89]:
final_data.isnull().sum()

unique_id                  0
truck_id                   0
route_id                   0
departure_date             0
estimated_arrival          0
delay                      0
route_avg_temp             0
route_avg_wind             0
route_description          0
route_avg_precip           0
route_avg_humidity         0
route_avg_visibility       0
route_avg_pressure         0
origin_id                  0
origin_avg_temp            0
origin_avg_wind            0
origin_description         0
origin_avg_precip          0
origin_avg_humidity        0
origin_avg_visibility      0
origin_avg_pressure        0
destination_id             0
dest_avg_temp              0
dest_avg_wind              0
dest_description           0
dest_avg_precip            0
dest_avg_humidity          0
dest_avg_visibility        0
dest_avg_pressure          0
avg_nov                    0
accident                   0
truck_age                  0
load_capacity_pounds     604
mileage_mpg                0
fuel_type     

# Load - Using Hopsworks (Feature Store) to load final data

Benifits of using feature store:
* Can be stored and shared within a team.
* We can have different versions.
* We can perform analystics.
* It assures data quality and monitoring.
* It can be integrated with ML platforms.

In [90]:
import hopsworks

In [91]:
api = '****************************************************'
project = hopsworks.login(api_key_value=api)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/792725


In [92]:
fs = project.get_feature_store()

2024-07-11 01:17:11,492 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-07-11 01:17:11,492 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [93]:
# We need to replace null values with 'unknown'

final_data['gender'].fillna('unknown',inplace=True)
final_data['driving_style'].fillna('unknown',inplace=True)


In [94]:
final_data.isnull().sum()

unique_id                  0
truck_id                   0
route_id                   0
departure_date             0
estimated_arrival          0
delay                      0
route_avg_temp             0
route_avg_wind             0
route_description          0
route_avg_precip           0
route_avg_humidity         0
route_avg_visibility       0
route_avg_pressure         0
origin_id                  0
origin_avg_temp            0
origin_avg_wind            0
origin_description         0
origin_avg_precip          0
origin_avg_humidity        0
origin_avg_visibility      0
origin_avg_pressure        0
destination_id             0
dest_avg_temp              0
dest_avg_wind              0
dest_description           0
dest_avg_precip            0
dest_avg_humidity          0
dest_avg_visibility        0
dest_avg_pressure          0
avg_nov                    0
accident                   0
truck_age                  0
load_capacity_pounds     604
mileage_mpg                0
fuel_type     

In [95]:
truck_final_fg = fs.get_or_create_feature_group(name='truck_final',
                                                version=1,
                                                description="Truck ETA Final Data",
                                                primary_key=['unique_id'],
                                                event_time='estimated_arrival',
                                                online_enabled=True)

In [96]:
truck_final_fg.insert(final_data)

Uploading Dataframe: 0.00% |          | Rows 0/12308 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: truck_final_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/792725/jobs/named/truck_final_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2227f971c70>, None)

In [97]:
truck_final_fg.statistics_config={'enabled':True,
                                 'correlations':True,
                                 'histograms':True}

In [98]:
truck_final_fg.update_statistics_config()

In [99]:
truck_final_fg.compute_statistics()

2024-07-11 01:17:36,390 WARNING: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).



Statistics(1720553092358)

In [100]:
hopsworks.logout()

Connection closed.
